In [1]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 1.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.2 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.1 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 2.8 MB/s eta 0:00:0000:0100:01


## Importing Libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time,os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import lightgbm as lgb
from flaml import AutoML
import polars as pl
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

## Importing dataset

In [2]:
# Read train and test datasets
train = pl.read_csv('train.csv')
test = pl.read_csv('test.csv')
submission=pl.read_csv('sample_submission.csv')

In [3]:
# Display the first few rows of each dataset
print("train:")
print(train.head())
print("\ntest :")
print(test.head())

train:
shape: (5, 3)
┌──────────┬───────────────────────────────────┬───────┐
│ essay_id ┆ full_text                         ┆ score │
│ ---      ┆ ---                               ┆ ---   │
│ str      ┆ str                               ┆ i64   │
╞══════════╪═══════════════════════════════════╪═══════╡
│ 000d118  ┆ Many people have car where they … ┆ 3     │
│ 000fe60  ┆ I am a scientist at NASA that is… ┆ 3     │
│ 001ab80  ┆ People always wish they had the … ┆ 4     │
│ 001bdc0  ┆ We all heard about Venus, the pl… ┆ 4     │
│ 002ba53  ┆ Dear, State Senator               ┆ 3     │
│          ┆                                   ┆       │
│          ┆ This is a l…                      ┆       │
└──────────┴───────────────────────────────────┴───────┘

test :
shape: (3, 2)
┌──────────┬───────────────────────────────────┐
│ essay_id ┆ full_text                         │
│ ---      ┆ ---                               │
│ str      ┆ str                               │
╞══════════╪════════

In [4]:
print(len(train))
print(len(test))

17307
3


In [5]:
# Summary statistics for numerical columns
print("\nSummary statistics for train dataset:")
print(train.describe())


Summary statistics for train dataset:
shape: (9, 4)
┌────────────┬──────────┬───────────────────────────────────┬──────────┐
│ describe   ┆ essay_id ┆ full_text                         ┆ score    │
│ ---        ┆ ---      ┆ ---                               ┆ ---      │
│ str        ┆ str      ┆ str                               ┆ f64      │
╞════════════╪══════════╪═══════════════════════════════════╪══════════╡
│ count      ┆ 17307    ┆ 17307                             ┆ 17307.0  │
│ null_count ┆ 0        ┆ 0                                 ┆ 0.0      │
│ mean       ┆ null     ┆ null                              ┆ 2.948402 │
│ std        ┆ null     ┆ null                              ┆ 1.044899 │
│ min        ┆ 000d118  ┆ "                                 ┆ 1.0      │
│            ┆          ┆                                   ┆          │
│            ┆          ┆ All of our development since …    ┆          │
│ max        ┆ fffed3e  ┆ ¨Wow look at that car it´s the k… ┆ 6.0      

In [6]:
print("Summary statistics for test dataset :")
print(test.describe())

Summary statistics for test dataset :
shape: (9, 3)
┌────────────┬──────────┬───────────────────────────────────┐
│ describe   ┆ essay_id ┆ full_text                         │
│ ---        ┆ ---      ┆ ---                               │
│ str        ┆ str      ┆ str                               │
╞════════════╪══════════╪═══════════════════════════════════╡
│ count      ┆ 3        ┆ 3                                 │
│ null_count ┆ 0        ┆ 0                                 │
│ mean       ┆ null     ┆ null                              │
│ std        ┆ null     ┆ null                              │
│ min        ┆ 000d118  ┆ I am a scientist at NASA that is… │
│ max        ┆ 001ab80  ┆ People always wish they had the … │
│ median     ┆ null     ┆ null                              │
│ 25%        ┆ null     ┆ null                              │
│ 75%        ┆ null     ┆ null                              │
└────────────┴──────────┴───────────────────────────────────┘


## Applying Vectorization and LGBM

In [7]:
#This code hrlp from that notebook-->[https://www.kaggle.com/code/davidjlochner/base-tfidf-lgbm/notebook]
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(min_df=.05)
train_tfid = vectorizer.fit_transform(train['full_text'])
test_tfid = vectorizer.transform(test['full_text'])

train_y = np.array(train['score'])

# Initialize AutoML for hyperparameter optimization
aml = AutoML()

# Fit AutoML to find the best hyperparameters
aml.fit(train_tfid, train_y, estimator_list=['lgbm'], task='classification', metric='macro_f1', time_budget=600)

# Retrieve the best hyperparameters found by AutoML
best_config = aml.best_config

# Initialize LGBMClassifier with the best hyperparameters found
model = lgb.LGBMClassifier(**best_config)

# Train the model
model.fit(train_tfid, train_y)

[flaml.automl.logger: 04-09 20:00:05] {1680} INFO - task = classification
[flaml.automl.logger: 04-09 20:00:05] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 04-09 20:00:06] {1789} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 04-09 20:00:06] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 04-09 20:00:06] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 04-09 20:00:06] {2346} INFO - Estimated sufficient time budget=8111s. Estimated necessary time budget=8s.
[flaml.automl.logger: 04-09 20:00:06] {2398} INFO -  at 0.9s,	estimator lgbm's best error=0.9072,	best estimator lgbm's best error=0.9072
[flaml.automl.logger: 04-09 20:00:06] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 04-09 20:00:07] {2398} INFO -  at 1.9s,	estimator lgbm's best error=0.9072,	best estimator lgbm's best error=0.9072
[flaml.automl.logger: 04-09 20:00:07] {2219} INFO - iteration 2, current learner lgbm
[

[LightGBM] [Warning] Unknown parameter: log_max_bin
[LightGBM] [Warning] Unknown parameter: log_max_bin
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 162435
[LightGBM] [Info] Number of data points in the train set: 17307, number of used features: 637
[LightGBM] [Info] Start training from score -2.626369
[LightGBM] [Info] Start training from score -1.298667
[LightGBM] [Info] Start training from score -1.013741
[LightGBM] [Info] Start training from score -1.483490
[LightGBM] [Info] Start training from score -2.881570
[LightGBM] [Info] Start training from score -4.709010


LGBMClassifier(learning_rate=0.10523786258143789, log_max_bin=8,
               min_child_samples=6, n_estimators=299, num_leaves=4,
               reg_alpha=0.06412115021351315, reg_lambda=0.5056770424681948)

In [10]:
import numpy as np

# Flatten the nested lists in the 'score' column
submission_flat = submission.explode('score')

# Write the flattened DataFrame to a CSV file
submission_flat.write_csv('submission.csv')

# Add insights
print("Submission generated successfully.")


Submission generated successfully.


In [11]:
submission_actual = pl.read_csv('submission.csv')
submission_actual

essay_id,score
str,i64
"""000d118""",3
"""000d118""",3
"""000d118""",4
"""000fe60""",3
"""000fe60""",3
"""000fe60""",4
"""001ab80""",3
"""001ab80""",3
"""001ab80""",4
